In [1]:
import yfinance as yf
import pandas as pd
from openai import OpenAI
from prophet import Prophet
import os
from dotenv import load_dotenv

## Generate basic information of company

In [3]:
def fetch_financial_data(ticker: str):
    stock = yf.Ticker(ticker)

    # Retrieve basic company info and historical prices
    info = stock.info
    hist = stock.history(period="1y")

    # Financial statements
    balance_sheet = stock.balance_sheet
    income_stmt = stock.financials

    # Calculate financial ratios: ROE (Return on Equity) and PE (Price-to-Earnings Ratio)
    try:
        net_income = income_stmt.loc["Net Income"].iloc[0]
        total_equity = balance_sheet.loc["Total Stockholder Equity"].iloc[0]
        roe = round(net_income / total_equity, 4) if total_equity != 0 else None
    except Exception:
        roe = None

    try:
        trailing_pe = info.get("trailingPE", None)
    except Exception:
        trailing_pe = None

    # Compile result summary
    summary = {
        "Company Name": info.get("longName", "N/A"),
        "Ticker": ticker,
        "Current Price": info.get("currentPrice", "N/A"),
        "PE Ratio": trailing_pe,
        "ROE": roe,
        "Market Cap": info.get("marketCap", "N/A"),
        "Industry": info.get("industry", "N/A")
    }

    return pd.DataFrame([summary]), hist.tail(5)


In [4]:
if __name__ == "__main__":
    ticker = input("Type in the stock symbol（ex. AAPL）：").upper()
    summary_df, recent_price = fetch_financial_data(ticker)
    yf_ticker = yf.Ticker(ticker)
    
    dividends = yf_ticker.dividends.tail(5)

    print("\n📊 Company Summary：")
    print(dividends)
    print("\n_________________")
    print(summary_df.T)

    print("\n📈 Last 5 days' closing prices：")
    print(recent_price[["Close"]])

Type in the stock symbol（ex. AAPL）： AAPL



📊 Company Summary：
Date
2024-05-10 00:00:00-04:00    0.25
2024-08-12 00:00:00-04:00    0.25
2024-11-08 00:00:00-05:00    0.25
2025-02-10 00:00:00-05:00    0.25
2025-05-12 00:00:00-04:00    0.26
Name: Dividends, dtype: float64

_________________
                                  0
Company Name             Apple Inc.
Ticker                         AAPL
Current Price               214.165
PE Ratio                   33.35903
ROE                            None
Market Cap            3198725390336
Industry       Consumer Electronics

📈 Last 5 days' closing prices：
                                Close
Date                                 
2025-07-22 00:00:00-04:00  214.399994
2025-07-23 00:00:00-04:00  214.149994
2025-07-24 00:00:00-04:00  213.759995
2025-07-25 00:00:00-04:00  213.880005
2025-07-28 00:00:00-04:00  214.139999


In [5]:
# Download 1-year price data and format it for Prophet forecasting
def get_price_data(ticker: str) -> pd.DataFrame:
    df = yf.download(ticker, period="1y")[['Close']].reset_index()
    df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)
    df['ds'] = pd.to_datetime(df['ds'])  # Ensure the 'ds' column is in datetime format
    df = df[['ds', 'y']]  # Keep only the columns required by Prophet
    df.columns = df.columns.droplevel(1)
    return df.dropna()    # Remove rows with missing values

# Estimate growth rate using ROE * (1 - payout ratio)
def get_growth_rate(ticker: str):
    yf_ticker = yf.Ticker(ticker)
    info = yf_ticker.info

    roe = info.get("returnOnEquity")      # e.g., 0.15
    payout = info.get("payoutRatio")      # e.g., 0.4

    if roe is None or payout is None:
        return "Unable to retrieve ROE or payout ratio"

    growth = roe * (1 - payout)
    return round(growth, 2)

# Forecast future stock prices using Prophet
def forecast_price_prophet(df: pd.DataFrame, n_days: int) -> pd.DataFrame:
    model = Prophet()
    model.fit(df)
    future = model.make_future_dataframe(periods=n_days)
    forecast = model.predict(future)
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Simple DCF (Discounted Cash Flow) valuation model
def simple_dcf(current_price, growth_rate, discount_rate, years=5):   
    fcf = current_price * 0.05  # Assume free cash flow is 5% of stock price
    npv = 0
    for t in range(1, years + 1):
        fcf *= (1 + growth_rate)
        npv += fcf / ((1 + discount_rate) ** t)
    terminal_value = fcf * (1 + growth_rate) / (discount_rate - growth_rate)
    npv += terminal_value / ((1 + discount_rate) ** years)
    return round(npv, 2)

# Simple DDM (Dividend Discount Model) valuation
def simple_ddm(ticker, growth_rate, discount_rate):
    yf_ticker = yf.Ticker(ticker)
    dividends = yf_ticker.dividends.tail(1).values[0]  # Get most recent dividend
    p = dividends * (1 + growth_rate) / (discount_rate - growth_rate)
    return round(p, 2)

# Determine trend (rising/falling) and valuation (under/overvalued)
def generate_result(ticker, forecast, dcf_value, current_price):
    final_price = forecast['yhat'].values[-1]
    trend = "上升" if final_price > current_price else "下降"
    valuation = "低估" if dcf_value > current_price else "高估"
    return trend, valuation

# Generate text summary of analysis results
def generate_summary(ticker, trend, dcf_value, ddm_value, valuation, current_price, growth_rate):
    return f"""🔍 Stock Analysis Report: {ticker}
📈 Forecasted Trend: {trend}
💰 DCF Valuation: {dcf_value} (Current Price: {round(current_price, 2)}, Growth Rate: {growth_rate}) → {valuation}
💰 DDM Valuation: {ddm_value} (Current Price: {round(current_price, 2)}, Growth Rate: {growth_rate}) → {valuation}
🧠 Recommendation: {'Consider Buying' if valuation == '低估' else 'Cautious Observation Recommended'}
"""

# Run full analysis pipeline
def run_analysis(ticker, discount_rate=0.08):
    df = get_price_data(ticker)
    forecast = forecast_price_prophet(df, n_days=30)
    current_price = df['y'].values[-1]
    growth_rate = get_growth_rate(ticker)
    if isinstance(growth_rate, str):
        print(growth_rate)
        return

    dcf_value = simple_dcf(current_price, growth_rate, discount_rate)
    ddm_value = simple_ddm(ticker, growth_rate, discount_rate)
    trend, valuation = generate_result(ticker, forecast, dcf_value, current_price)
    summary = generate_summary(ticker, trend, dcf_value, ddm_value, valuation, current_price, growth_rate)
    print(summary)

# Generate a concise investment opinion in Chinese using OpenAI API
def generate_investment_opinion_openai(ticker, trend, valuation, client):
    # Create the prompt for the AI model based on trend and valuation
    prompt = f"""
You are a senior investment advisor. Based on the information below, please write a brief investment recommendation for the stock {ticker}.

Trend Forecast: {trend}
Valuation Status: {valuation}

Please provide a professional and cautious investment opinion (based on the forecast and valuation), limited to 2-3 sentences and no more than 100 characters.
    """.strip()

    try:
        # Send the prompt to the OpenAI chat model
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a professional Chinese-language equity investment analyst."},
                {"role": "user", "content": prompt}
            ]
        )
        # Return the generated message content
        return response.choices[0].message.content.strip()

    except Exception as e:
        # Return error message if generation fails
        return f"(⚠️ Generation failed: {e})"

In [6]:
def main():
    load_dotenv()

    ticker = input("Type in the stock symbol（ex. AAPL）：").strip()
    n_days = int(input("How many days to predict："))

    df = get_price_data(ticker)
    forecast = forecast_price_prophet(df, n_days)
    current_price = df['y'].values[-1]
    growth_rate = 0.05
    discount_rate = 0.08
    dcf_value = simple_dcf(current_price, growth_rate, discount_rate)
    ddm_value = simple_ddm(ticker, growth_rate, discount_rate)
    trend, valuation = generate_result(ticker, forecast, dcf_value, current_price)
    summary = generate_summary(ticker, trend, dcf_value, ddm_value, valuation, current_price, growth_rate)

    api_key = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)

    analysis = generate_investment_opinion_openai(ticker, trend, valuation, client)

    print(f"\n{ticker} next {n_days} days prediction：")
    print(forecast.tail(n_days))
    print(summary)
    print(analysis)

if __name__ == "__main__":
    main()


輸入股票代號（如 AAPL）： AAPL
預測未來幾天： 7


C:\Users\kkiwi\AppData\Local\Temp\ipykernel_32012\3824065370.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y")[['Close']].reset_index()
[*********************100%***********************]  1 of 1 completed
22:01:15 - cmdstanpy - INFO - Chain [1] start processing
22:01:15 - cmdstanpy - INFO - Chain [1] done processing



AAPL 未來 7 天預測股價：
            ds        yhat  yhat_lower  yhat_upper
250 2025-07-29  208.638778  198.467915  218.789476
251 2025-07-30  208.642880  198.864771  218.628945
252 2025-07-31  208.966077  198.263294  218.977807
253 2025-08-01  208.782687  198.624845  219.317745
254 2025-08-02  209.259271  200.195012  218.973422
255 2025-08-03  209.337388  198.807483  218.585283
256 2025-08-04  208.834028  199.437891  219.077283
🔍 Stock Analysis Report: AAPL
📈 Forecasted Trend: 下降
💰 DCF Valuation: 374.75 (Current Price: 214.15, Growth Rate: 0.05) → 低估
💰 DDM Valuation: 9.1 (Current Price: 214.15, Growth Rate: 0.05) → 低估
🧠 Recommendation: Consider Buying

Given the downward trend forecast and undervaluation status, proceed with caution and consider adding AAPL to your watchlist for a potential buying opportunity.
